<a href="https://colab.research.google.com/github/LM1997610/NN/blob/main/HW_1_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

import matplotlib.pyplot as plt

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [3]:
train_images = tf.cast(train_images, tf.float32)/ 255.0
test_images = tf.cast(test_images, tf.float32)/ 255.0

In [4]:
def add_conv_block(model, n_filters):

  model.add(keras.layers.Conv2D(filters = n_filters,                  # conv_layer
                                kernel_size = (3, 3),
                                padding = "same"))

  model.add(keras.layers.BatchNormalization())                       # batch_normalization
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))  # pool_layer
  model.add(keras.layers.Activation('relu'))                         # activation
  model.add(keras.layers.Dropout(0.3))                               # drop_out

  return model

In [5]:
hidden_layers =  [128, 512, 512, 512, 512]

model = keras.Sequential()

for i in hidden_layers:
  model = add_conv_block(model, i)

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(10))


In [6]:
model.build(input_shape=(256, 32,32,3))
# model.summary()

In [7]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = keras.metrics.SparseCategoricalAccuracy()
opt = keras.optimizers.Adam(learning_rate = 2e-3, weight_decay= 0.001)

In [ ]:
model.compile(optimizer= opt,
              loss = cross_entropy,
              metrics = ["accuracy"])

history = model.fit(train_images, train_labels, epochs=15,
                    validation_data=(test_images, test_labels), batch_size = 300 )

Epoch 1/15
167/167 [==============================] - 49s 184ms/step - loss: 1.8106 - accuracy: 0.3520 - val_loss: 4.0191 - val_accuracy: 0.1192
Epoch 2/15
167/167 [==============================] - 24s 145ms/step - loss: 1.1659 - accuracy: 0.5752 - val_loss: 2.9931 - val_accuracy: 0.1727
Epoch 3/15
167/167 [==============================] - 24s 146ms/step - loss: 0.9056 - accuracy: 0.6773 - val_loss: 2.1762 - val_accuracy: 0.3826
Epoch 4/15
167/167 [==============================] - 25s 148ms/step - loss: 0.7697 - accuracy: 0.7282 - val_loss: 1.5589 - val_accuracy: 0.5454
Epoch 5/15
167/167 [==============================] - 24s 146ms/step - loss: 0.6725 - accuracy: 0.7651 - val_loss: 1.2893 - val_accuracy: 0.5980
Epoch 6/15
156/167 [===========================>..] - ETA: 1s - loss: 0.6009 - accuracy: 0.7900

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(11, 3))
fig.tight_layout(pad=4, w_pad = 6.5)

ax[0].plot(history.history['loss'], label='train_loss', color = "red")
ax[0].plot(history.history['val_loss'], label='val_loss', color = "green")

ax[0].set_title('Loss History\n', fontsize=14)
ax[0].set_xlabel('\n Epochs')
ax[0].set_ylabel('Loss')
#ax[0].set_xticks(list(range(1, num_epochs+1, 3)))
ax[0].grid(linewidth=0.4)
ax[0].legend(loc="best", prop={'size': 8})


ax[1].plot(history.history['sparse_categorical_accuracy'], label='train_accuracy', color = "red")
ax[1].plot(history.history['val_sparse_categorical_accuracy'], label = 'val_accuracy', color = "green")
ax[1].set_title('Classification Accuracy History\n', fontsize=14)
ax[1].set_xlabel('\n Epochs')
ax[1].set_ylabel('Classification accuracy')
ax[1].grid(linewidth=0.4)

#ax[1].set_xticks(list(range(1, num_epochs+1, 3)))
ax[1].legend(loc ="lower right", prop={'size': 8})

#plt.savefig("history_plot.png")
plt.show()

In [ ]:
cifar10_class_names = {0: "Plane", 1: "Car", 2: "Bird", 3: "Cat", 4: "Deer",
                       5: "Dog", 6: "Frog", 7: "Horse", 8: "Boat", 9: "Truck"}

In [ ]:
import numpy as np

In [ ]:
def show_stuff(index):

  with tf.GradientTape() as tape:

    tensor_input = tf.convert_to_tensor(np.expand_dims(test_images[index], axis=0))
    tape.watch(tensor_input)
    pred = model(tensor_input)
    pred_class = tf.argmax(pred[0])
    class_channel = pred[:,pred_class]

  print(f" True Label: {test_labels[index][0]} - {cifar10_class_names[test_labels[index].item()]}")
  print(f" Prediction: {pred_class.numpy()} - {cifar10_class_names[pred_class.numpy()]}")

  grads = tape.gradient(class_channel, tensor_input)
  g = tf.squeeze(grads).numpy()
  g = ((g - g.min()) * (1/(g.max() - g.min()) * 255)).astype('uint8')

  fig, ax = plt.subplots(1, 2, figsize=(5, 5))

  ax[0].imshow(g);
  ax[0].axis('off')

  ax[1].imshow(test_images[index]);
  ax[1].axis('off')

  plt.show()


In [ ]:
show_stuff(14)

In [ ]:
print(f" True Label: {cifar10_class_names[test_labels[2].item()]} \n")

fig = plt.figure(figsize=(2.5, 2.5))
plt.imshow(test_images[2]);
plt.gca().axis('off')
plt.show()

In [ ]:
import numpy as np

In [ ]:
model.layers[-1].activation = None

In [ ]:
with tf.GradientTape() as tape:

  tensor_input = tf.convert_to_tensor(np.expand_dims(test_images[2], axis=0))
  tape.watch(tensor_input)
  pred = model(tensor_input)
  pred_class = tf.argmax(pred[0])
  class_channel = pred[:,pred_class]


In [ ]:
grads = tape.gradient(class_channel, tensor_input)

In [ ]:
fig = plt.figure(figsize=(2.5, 2.5))
plt.imshow(this_img);
plt.gca().axis('off')
plt.show()

In [ ]:
print(f" True Label: {cifar10_class_names[test_labels[2].item()]} \n")

fig = plt.figure(figsize=(2.5, 2.5))
plt.imshow(test_images[2]);
plt.gca().axis('off')
plt.show()

In [ ]:
g = tf.squeeze(grads).numpy()
g = ((g - g.min()) * (1/(g.max() - g.min()) * 255)).astype('uint8')

In [ ]:
h = tf.maximum(g,0) / tf.math.reduce_max(g)
h = h.numpy()

In [ ]:
plt.matshow(h)
plt.show()